<a href="https://colab.research.google.com/github/Aakash007-ai/Labs/blob/master/Edge_detection_using_dwt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math

import cv2
import numpy as np
import scipy.ndimage
from matplotlib import pyplot as plt


In [2]:
direction_rows = 0
direction_columns = 1

scale_order = 2

noise_gaussian = "Gaussian Noise"
noise_salt_pepper = "Salt & Pepper Noise"

filter_haar = "Haar Filter"
filter_coiflet = "Coiflet Filter"

threshold_final = 1e5

threshold_original_haar = 15
threshold_original_coiflet = 14

threshold_gaussian_haar = 18
threshold_gaussian_coiflet = 16

threshold_snp_haar = 15
threshold_snp_coifflet = 13

In [3]:
def generateNoise(img_input, noise_type, level_val_1, level_val_2):
    if noise_type == noise_gaussian:
        height, width = img_input.shape

        mean = level_val_1
        var = level_val_2
        sigma = var ** 0.5  # Sample 20

        # Gaussian-distributed additive noise
        gauss = np.random.normal(mean, sigma, (height, width))
        gauss = gauss.reshape(height, width)
        img_gaussian_noise = img_input + gauss

        return img_gaussian_noise

    elif noise_type == noise_salt_pepper:
        img_salt_pepper_noise = img_input.copy()

        salt_vs_pepper = level_val_1
        amount = level_val_2

        # Salt mode : Replaces random pixels with 1
        num_salt = np.ceil(amount * img_input.size * salt_vs_pepper)
        pixels = [np.random.randint(0, i - 1, int(num_salt))
                  for i in img_input.shape]
        img_salt_pepper_noise[pixels] = 1

        # Pepper mode : Replaces random pixels with 0
        num_pepper = np.ceil(amount * img_input.size * (1. - salt_vs_pepper))
        pixels = [np.random.randint(0, i - 1, int(num_pepper))
                  for i in img_input.shape]
        img_salt_pepper_noise[pixels] = 0

        return img_salt_pepper_noise


In [4]:
def downSample(img_input, direction):
    if direction == direction_rows:
        h, w = img_input.shape
        img_output = np.zeros((h // 2, w))   #c1 h//2
        i = 0
        new = i
        while i < h:
            img_output[new, :] = img_input[i, :]
            new = new + 1
            i = i + 2

        return img_output

    elif direction == direction_columns:
        h, w = img_input.shape
        img_output = np.zeros((h, w // 2))  #c2
        i = 0
        new = i
        while i < w:
            img_output[:, new] = img_input[:, i]
            new = new + 1
            i = i + 2

        return img_output


In [5]:
def upSample(img_input, order):
    img_output = scipy.ndimage.zoom(img_input, order)
    return img_output

In [6]:
def haarConvolveRowsLowPass(img_input):
    img_input_pad = cv2.copyMakeBorder(img_input, 0, 0, 0, 1, cv2.BORDER_CONSTANT, value=0)

    height, width = img_input_pad.shape

    print("haarConvolveRowsLowPass : ")
    print(img_input_pad.shape)

    img_haar_convolved = np.zeros((height, width))

    for x in range(0, height):
        for y in range(0, width - 1):
            img_haar_convolved[x, y] = (img_input_pad[x, y] / math.sqrt(2)) + (
                img_input_pad[x, y + 1] / math.sqrt(2))

    return img_haar_convolved[0:height, 0:width - 1]

In [7]:
def haarConvolveColumnsLowPass(img_input):
    img_input_pad = cv2.copyMakeBorder(img_input, 0, 1, 0, 0, cv2.BORDER_CONSTANT, value=0)

    height, width = img_input_pad.shape

    print("haarConvolveColumnsLowPass : ")
    print(img_input_pad.shape)

    img_haar_convolved = np.zeros((height, width))

    for y in range(0, width):
        for x in range(0, height - 1):
            img_haar_convolved[x, y] = (img_input_pad[x, y] / math.sqrt(2)) + (
                img_input_pad[x + 1, y] / math.sqrt(2))

    return img_haar_convolved[0:height - 1, 0:width]


In [8]:
def haarConvolveRowsHighPass(img_input):
    img_input_pad = cv2.copyMakeBorder(img_input, 0, 0, 0, 1, cv2.BORDER_CONSTANT, value=0)

    height, width = img_input_pad.shape

    print("haarConvolveRowsHighPass : ")
    print(img_input_pad.shape)

    img_haar_convolved = np.zeros((height, width))

    for x in range(0, height):
        for y in range(0, width - 1):
            img_haar_convolved[x, y] = -(img_input_pad[x, y] / math.sqrt(2)) + (
                img_input_pad[x, y + 1] / math.sqrt(2))

    return img_haar_convolved[0:height, 0:width - 1]


In [10]:
def haarConvolveColumnsHighPass(img_input):
    img_input_pad = cv2.copyMakeBorder(img_input, 0, 1, 0, 0, cv2.BORDER_CONSTANT, value=0)

    height, width = img_input_pad.shape

    print("haarConvolveColumnsHighPass : ")
    print(img_input_pad.shape)

    img_haar_convolved = np.zeros((height, width))

    for y in range(0, width):
        for x in range(0, height - 1):
            img_haar_convolved[x, y] = -(img_input_pad[x, y] / math.sqrt(2)) + (
                img_input_pad[x + 1, y] / math.sqrt(2))

    return img_haar_convolved[0:height - 1, 0:width]

In [11]:
def coifletConvolveRowsLowPass(img_input):
    # -0.0157   -0.0727    0.3849      0.8526    0.3379   -0.0727
    img_input_pad = cv2.copyMakeBorder(img_input, 0, 0, 2, 3, cv2.BORDER_CONSTANT, value=0)

    height, width = img_input_pad.shape

    print("coifletConvolveRowsLowPass : ")
    print(img_input_pad.shape)

    img_coiflet_convolved = np.zeros((height, width))

    for x in range(0, height):
        for y in range(2, width - 3):
            img_coiflet_convolved[x, y] = img_input_pad[x, y - 2] * (-0.0157) + img_input_pad[x, y - 1] * (-0.0727) + \
                                          img_input_pad[x, y] * (0.3849) + img_input_pad[x, y + 1] * (0.8526) + \
                                          img_input_pad[x, y + 2] * (0.3379) + img_input_pad[x, y + 3] * (-0.0727)

    return img_coiflet_convolved[0:height, 2:width - 3]


In [12]:
def coifletConvolveColumnsLowPass(img_input):
    # -0.0157   -0.0727    0.3849      0.8526    0.3379   -0.0727
    img_input_pad = cv2.copyMakeBorder(img_input, 2, 3, 0, 0, cv2.BORDER_CONSTANT, value=0)

    height, width = img_input_pad.shape

    print("coifletConvolveColumnsLowPass : ")
    print(img_input_pad.shape)

    img_coiflet_convolved = np.zeros((height, width))

    for x in range(2, height - 3):
        for y in range(0, width):
            img_coiflet_convolved[x, y] = img_input_pad[x - 2, y] * (-0.0157) + img_input_pad[x - 1, y] * (-0.0727) + \
                                          img_input_pad[x, y] * (0.3849) + img_input_pad[x + 1, y] * (0.8526) + \
                                          img_input_pad[x + 2, y] * (0.3379) + img_input_pad[x + 3, y] * (-0.0727)

    return img_coiflet_convolved[2:height - 3, 0:width]


In [13]:
def coifletConvolveRowsHighPass(img_input):
    # 0.0727    0.3379   -0.8526    0.3849    0.0727   -0.0157; High
    img_input_pad = cv2.copyMakeBorder(img_input, 0, 0, 2, 3, cv2.BORDER_CONSTANT, value=0)

    height, width = img_input_pad.shape

    print("coifletConvolveRowsHighPass : ")
    print(img_input_pad.shape)

    img_coiflet_convolved = np.zeros((height, width))

    for x in range(0, height):
        for y in range(2, width - 3):
            img_coiflet_convolved[x, y] = img_input_pad[x, y - 2] * (0.0727) + img_input_pad[x, y - 1] * (0.3379) + \
                                          img_input_pad[x, y] * (-0.8526) + img_input_pad[x, y + 1] * (0.3849) + \
                                          img_input_pad[x, y + 2] * (0.0727) + img_input_pad[x, y + 3] * (-0.0157)

    return img_coiflet_convolved[0:height, 2:width - 3]


In [14]:
def coifletConvolveColumnsHighPass(img_input):
    # 0.0727    0.3379   -0.8526    0.3849    0.0727   -0.0157; High
    img_input_pad = cv2.copyMakeBorder(img_input, 2, 3, 0, 0, cv2.BORDER_CONSTANT, value=0)

    height, width = img_input_pad.shape

    print("coifletConvolveColumnsHighPass : ")
    print(img_input_pad.shape)

    img_coiflet_convolved = np.zeros((height, width))

    for x in range(2, height - 3):
        for y in range(0, width):
            img_coiflet_convolved[x, y] = img_input_pad[x - 2, y] * (0.0727) + img_input_pad[x - 1, y] * (0.3379) + \
                                          img_input_pad[x, y] * (-0.8526) + img_input_pad[x + 1, y] * (0.3849) + \
                                          img_input_pad[x + 2, y] * (0.0727) + img_input_pad[x + 3, y] * (-0.0157)
    return img_coiflet_convolved[2:height - 3, 0:width]


In [15]:
def performLL(img_input, filter_wavelet):
    if filter_wavelet == filter_haar:
        img_L_rows = haarConvolveRowsLowPass(img_input)
        img_L_rows_downsampled = downSample(img_L_rows, direction_rows)

        img_L_columns = haarConvolveColumnsLowPass(img_L_rows_downsampled)
        img_L_columns_downsampled = downSample(img_L_columns, direction_columns)

        return img_L_columns_downsampled

    elif filter_wavelet == filter_coiflet:
        img_L_rows = coifletConvolveRowsLowPass(img_input)
        img_L_rows_downsampled = downSample(img_L_rows, direction_rows)

        img_L_columns = coifletConvolveColumnsLowPass(img_L_rows_downsampled)
        img_L_columns_downsampled = downSample(img_L_columns, direction_columns)

        return img_L_columns_downsampled


In [16]:
def performLH(img_input, filter_wavelet):
    if filter_wavelet == filter_haar:
        img_L_rows = haarConvolveRowsLowPass(img_input)
        img_L_rows_downsampled = downSample(img_L_rows, direction_rows)

        img_H_columns = haarConvolveColumnsHighPass(img_L_rows_downsampled)
        img_H_columns_downsampled = downSample(img_H_columns, direction_columns)

        return img_H_columns_downsampled

    elif filter_wavelet == filter_coiflet:
        img_L_rows = coifletConvolveRowsLowPass(img_input)
        img_L_rows_downsampled = downSample(img_L_rows, direction_rows)

        img_H_columns = coifletConvolveColumnsHighPass(img_L_rows_downsampled)
        img_H_columns_downsampled = downSample(img_H_columns, direction_columns)

        return img_H_columns_downsampled


In [17]:
def performHL(img_input, filter_wavelet):
    if filter_wavelet == filter_haar:
        img_H_rows = haarConvolveRowsHighPass(img_input)
        img_H_rows_downsampled = downSample(img_H_rows, direction_rows)

        img_L_columns = haarConvolveColumnsLowPass(img_H_rows_downsampled)
        img_L_columns_downsampled = downSample(img_L_columns, direction_columns)

        return img_L_columns_downsampled

    elif filter_wavelet == filter_coiflet:
        img_H_rows = coifletConvolveRowsHighPass(img_input)
        img_H_rows_downsampled = downSample(img_H_rows, direction_rows)

        img_L_columns = coifletConvolveColumnsLowPass(img_H_rows_downsampled)
        img_L_columns_downsampled = downSample(img_L_columns, direction_columns)

        return img_L_columns_downsampled


In [18]:
def performHH(img_input, filter_wavelet):
    if filter_wavelet == filter_haar:
        img_H_rows = haarConvolveRowsHighPass(img_input)
        img_H_rows_downsampled = downSample(img_H_rows, direction_rows)

        img_H_columns = haarConvolveColumnsHighPass(img_H_rows_downsampled)
        img_H_columns_downsampled = downSample(img_H_columns, direction_columns)

        return img_H_columns_downsampled

    elif filter_wavelet == filter_coiflet:
        img_H_rows = coifletConvolveRowsHighPass(img_input)
        img_H_rows_downsampled = downSample(img_H_rows, direction_rows)

        img_H_columns = coifletConvolveColumnsHighPass(img_H_rows_downsampled)
        img_H_columns_downsampled = downSample(img_H_columns, direction_columns)

        return img_H_columns_downsampled


In [19]:
def pipeline(img_input, filter_wavelet, noise, threshold_subband):
    # ------------------------ Step 1 : Applying Wavelet Filter, Threshold and Down Sampling ---------------------------
    # Level 1 : Perform filters LL, LH, HL and HH on Input Image of size [M, N] and generate [M/2, N/2] size Image
    img_LL_2 = performLL(img_input, filter_wavelet)
    img_LH_2 = performLH(img_input, filter_wavelet)
    img_HL_2 = performHL(img_input, filter_wavelet)
    img_HH_2 = performHH(img_input, filter_wavelet)

    printIntermediateResults(img_LL_2, img_LH_2, img_HL_2, img_HH_2, filter_wavelet, noise, "[M/2, N/2] Before Threshold")

    # Applying threshold to LH, HL and HH subbands
    img_LH_2 = performThresholding(img_LH_2, threshold_subband, False)
    img_HL_2 = performThresholding(img_HL_2, threshold_subband, False)
    img_HH_2 = performThresholding(img_HH_2, threshold_subband, False)

    printIntermediateResults(img_LL_2, img_LH_2, img_HL_2, img_HH_2, filter_wavelet, noise, "[M/2, N/2] After Threshold")

    # Level 2 : Perform filters LL, LH, HL and HH on LL Image of size [M/2, N/2] and generate [M/4, N/4] size Image
    img_LL_4 = performLL(img_LL_2, filter_wavelet)
    img_LH_4 = performLH(img_LL_2, filter_wavelet)
    img_HL_4 = performHL(img_LL_2, filter_wavelet)
    img_HH_4 = performHH(img_LL_2, filter_wavelet)

    # printIntermediateResults(img_LL_4, img_LH_4, img_HL_4, img_HH_4, filter_wavelet, noise, "[M/4, N/4] Before Threshold")

    # Applying threshold to LH, HL and HH subbands
    img_LH_4 = performThresholding(img_LH_4, threshold_subband, False)
    img_HL_4 = performThresholding(img_HL_4, threshold_subband, False)
    img_HH_4 = performThresholding(img_HH_4, threshold_subband, False)

    # printIntermediateResults(img_LL_4, img_LH_4, img_HL_4, img_HH_4, filter_wavelet, noise, "[M/4, N/4] After Threshold")

    # Level 3 : Perform filters LL, LH, HL and HH on LL Image of size [M/4, N/4] and generate [M/8, N/8] size Image
    img_LL_8 = performLL(img_LL_4, filter_wavelet)
    img_LH_8 = performLH(img_LL_4, filter_wavelet)
    img_HL_8 = performHL(img_LL_4, filter_wavelet)
    img_HH_8 = performHH(img_LL_4, filter_wavelet)

    # printIntermediateResults(img_LL_8, img_LH_8, img_HL_8, img_HH_8, filter_wavelet, noise,"[M/8, N/8] Before Threshold")

    # Applying threshold to LH, HL and HH subbands
    img_LH_8 = performThresholding(img_LH_8, threshold_subband, False)
    img_HL_8 = performThresholding(img_HL_8, threshold_subband, False)
    img_HH_8 = performThresholding(img_HH_8, threshold_subband, False)

    # printIntermediateResults(img_LL_8, img_LH_8, img_HL_8, img_HH_8, filter_wavelet, noise, "[M/8, N/8] After Threshold")

    # Level 4 : Perform filters LL, LH, HL and HH on LL Image of size [M/8, N/8] and generate [M/16, N/16] size Image
    img_LL_16 = performLL(img_LL_8, filter_wavelet)
    img_LH_16 = performLH(img_LL_8, filter_wavelet)
    img_HL_16 = performHL(img_LL_8, filter_wavelet)
    img_HH_16 = performHH(img_LL_8, filter_wavelet)

    # printIntermediateResults(img_LL_16, img_LH_16, img_HL_16, img_HH_16, filter_wavelet, noise, "[M/16, N/16] Before Threshold")

    # Applying threshold to LH, HL and HH subbands
    img_LH_16 = performThresholding(img_LH_16, threshold_subband, False)
    img_HL_16 = performThresholding(img_HL_16, threshold_subband, False)
    img_HH_16 = performThresholding(img_HH_16, threshold_subband, False)

    # printIntermediateResults(img_LL_16, img_LH_16, img_HL_16, img_HH_16, filter_wavelet, noise, "[M/16, N/16] After Threshold")

    # --------------------------------- Step 2 : Up Sampling and Image Matrix Multiplication ---------------------------
    # Blowing Up Level 4 to Level 3 : Up Sampling LH, HL and HH Images of size [M/16, N/16] to size [M/8, N/8]
    img_LH_16_blown = upSample(img_LH_16, scale_order)
    img_HL_16_blown = upSample(img_HL_16, scale_order)
    img_HH_16_blown = upSample(img_HH_16, scale_order)

    # printIntermediateResults(img_LL_16, img_LH_16_blown, img_HL_16_blown, img_HH_16_blown, filter_wavelet, noise, "[M/16, N/16] Blown to [M/8, N/8]")

    # Element wise Multiplication Level 4 blown up Images of size [M/8, N/8] with original Level 3 Images of size [M/8, N/8]
    img_LH_8_new = img_LH_16_blown * img_LH_8
    img_HL_8_new = img_HL_16_blown * img_HL_8
    img_HH_8_new = img_HH_16_blown * img_HH_8

    # printIntermediateResults(img_LL_8, img_LH_8_new, img_HL_8_new, img_HH_8_new, filter_wavelet, noise, "[M/8, N/8] After Multiplying")

    # Blowing Up Level 3 to Level 2 : Up Sampling LH, HL and HH Images of size [M/8, N/8] to size [M/4, N/4]
    img_LH_8_blown = upSample(img_LH_8_new, scale_order)
    img_HL_8_blown = upSample(img_HL_8_new, scale_order)
    img_HH_8_blown = upSample(img_HH_8_new, scale_order)

    # printIntermediateResults(img_LL_8, img_LH_8_blown, img_HL_8_blown, img_HH_8_blown, filter_wavelet, noise, "[M/8, N/8] Blown to [M/4, N/4]")

    # Element wise Multiplication Level 3 blown up Images of size [M/4, N/4] with original Level 2 Images of size [M/4, N/4]
    img_LH_4_new = img_LH_8_blown * img_LH_4
    img_HL_4_new = img_HL_8_blown * img_HL_4
    img_HH_4_new = img_HH_8_blown * img_HH_4

    # printIntermediateResults(img_LL_4, img_LH_4_new, img_HL_4_new, img_HH_4_new, filter_wavelet, noise, "[M/4, N/4] After Multiplying")

    # Blowing Up Level 2 to Level 1 : Up Sampling LH, HL and HH Images of size [M/4, N/4] to size [M/2, N/2]
    img_LH_4_blown = upSample(img_LH_4_new, scale_order)
    img_HL_4_blown = upSample(img_HL_4_new, scale_order)
    img_HH_4_blown = upSample(img_HH_4_new, scale_order)

    # printIntermediateResults(img_LL_4, img_LH_4_blown, img_HL_4_blown, img_HH_4_blown, filter_wavelet, noise, "[M/4, N/4] Blown to [M/2, N/2]")

    # Element wise Multiplication Level 2 blown up Images of size [M/2, N/2] with original Level 1 Images of size [M/2, N/2]
    img_LH_2_new = img_LH_4_blown * img_LH_2
    img_HL_2_new = img_HL_4_blown * img_HL_2
    img_HH_2_new = img_HH_4_blown * img_HH_2

    # printIntermediateResults(img_LL_2, img_LH_2_new, img_HL_2_new, img_HH_2_new, filter_wavelet, noise, "[M/2, N/2] After Multiplying")

    print (img_LH_2_new.shape)

    # --------------------------------------- Step 3 : Generating Final Edge map ---------------------------------------
    img_LH_final = np.multiply(img_LH_2_new, img_LH_2_new)
    img_HL_final = np.multiply(img_HL_2_new, img_HL_2_new)
    img_HH_final = np.multiply(img_HH_2_new, img_HH_2_new)
    img_final_edges = np.sqrt(img_LH_final + img_HL_final + img_HH_final)

    return True, img_final_edges
